In [114]:
from IPython import display

In [115]:
try:
    import nbformat
except:
    !pip install nbformat
display.clear_output()

In [116]:
%run ../model/train.ipynb

+ action: train_feat01
+ feat_path: ../../exps/featbase_18102025/data.npz
+ seed: 42
+ exp_dir: c:\Users\LOQ\Downloads\tmp\pocess_challenge1_titanic\process\exps
+ exp_name: trainbase_18102025
+ data_dir: c:\Users\LOQ\Downloads\tmp\pocess_challenge1_titanic\data
+ verbose: True
+ save_dir: c:\Users\LOQ\Downloads\tmp\pocess_challenge1_titanic\process\exps\trainbase_18102025
---------- information ----------
train-col: {'Survived', 'Cabin', 'PassengerId', 'Pclass', 'Age', 'Parch', 'Embarked', 'Fare', 'Ticket', 'Name', 'Sex', 'SibSp'}
test-col: {'Cabin', 'PassengerId', 'Pclass', 'Age', 'Parch', 'Embarked', 'Fare', 'Ticket', 'Name', 'Sex', 'SibSp'}
Union: {'Cabin', 'PassengerId', 'Pclass', 'Age', 'Parch', 'Embarked', 'Fare', 'Ticket', 'Name', 'Sex', 'SibSp'}
Difference: {'Survived'}
Runing ... [train_feat01]
Đã lưu DataFrame với đầy đủ thông tin cột


In [117]:
!jupyter nbconvert  ../model/train.ipynb --to html 

[NbConvertApp] Converting notebook ../model/train.ipynb to html
[NbConvertApp] Writing 373677 bytes to ..\model\train.html


# Import lib


In [130]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_auc_score, roc_curve
import joblib

data = np.load("../../exps/trainbase_18102025/data.npz", allow_pickle=True)

In [119]:
with np.load('../../exps/trainbase_18102025/data.npz') as z:
    print(z.files)          # shows the list of keys present

['train_data', 'test_data', 'train_columns', 'test_columns']


In [120]:
# Lấy dữ liệu train và test
df_train = pd.DataFrame(data = data['train_data'], columns= data['train_columns'])
df_test = pd.DataFrame(data['test_data'], columns=df_train.drop(columns=['Output']).columns)


In [121]:
df_train

,Sex,Age,Fare,Pclass,SibSp,Parch,Embarked,Output
0,1.0,22.000000,7.2500,3.0,1.0,0.0,3.0,0.0
1,0.0,38.000000,71.2833,1.0,1.0,0.0,1.0,1.0
2,0.0,26.000000,7.9250,3.0,0.0,0.0,3.0,1.0
3,0.0,35.000000,53.1000,1.0,1.0,0.0,3.0,1.0
4,1.0,35.000000,8.0500,3.0,0.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...
886,1.0,27.000000,13.0000,2.0,0.0,0.0,3.0,0.0
887,0.0,19.000000,30.0000,1.0,0.0,0.0,3.0,1.0
888,0.0,29.699118,23.4500,3.0,1.0,2.0,3.0,0.0
889,1.0,26.000000,30.0000,1.0,0.0,0.0,1.0,1.0


In [122]:
df_test

,Sex,Age,Fare,Pclass,SibSp,Parch,Embarked
0,1.0,34.50000,7.8292,3.0,0.0,0.0,2.0
1,0.0,47.00000,7.0000,3.0,1.0,0.0,3.0
2,1.0,62.00000,9.6875,2.0,0.0,0.0,2.0
3,1.0,27.00000,8.6625,3.0,0.0,0.0,3.0
4,0.0,22.00000,12.2875,3.0,1.0,1.0,3.0
...,...,...,...,...,...,...,...
413,1.0,30.27259,8.0500,3.0,0.0,0.0,3.0
414,0.0,39.00000,108.9000,1.0,0.0,0.0,1.0
415,1.0,38.50000,7.2500,3.0,0.0,0.0,3.0
416,1.0,30.27259,8.0500,3.0,0.0,0.0,3.0


In [ ]:
X_train = df_train.drop(columns=['Output', 'PassengerId'], axis=1)
y_train = df_train['Output']

In [124]:
X_test = df_test

# Logistic Regression


In [125]:
# Tạo pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(random_state=42, max_iter=1000))
])

# Thiết lập hyperparameters
param_grid = {
    'logreg__C': [0.01, 0.1, 1, 10, 100],
    'logreg__penalty': ['l1', 'l2'],
    'logreg__solver': ['liblinear']
}

# Sử dụng Stratified K-Fold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search với multiple scoring metrics
grid_search = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=cv, 
    scoring={'accuracy': 'accuracy', 'f1': 'f1', 'roc_auc': 'roc_auc'},
    refit='accuracy',  # Chọn metric để chọn best model
    n_jobs=-1,
    verbose=1
)

# Train model
print("\nBắt đầu training với GridSearch...")
grid_search.fit(X_train, y_train)

# Kết quả
print("\nBest parameters:", grid_search.best_params_)
print("Best cross-validation accuracy: {:.4f}".format(grid_search.best_score_))

# Lấy tất cả kết quả cross-validation
cv_results = grid_search.cv_results_
print("\nCross-validation results:")
for i in range(len(cv_results['params'])):
    print(f"Params: {cv_results['params'][i]}")
    print(f"  Accuracy: {cv_results['mean_test_accuracy'][i]:.4f} (+/- {cv_results['std_test_accuracy'][i]:.4f})")
    print(f"  F1: {cv_results['mean_test_f1'][i]:.4f} (+/- {cv_results['std_test_f1'][i]:.4f})")
    print(f"  ROC AUC: {cv_results['mean_test_roc_auc'][i]:.4f} (+/- {cv_results['std_test_roc_auc'][i]:.4f})")


Bắt đầu training với GridSearch...
Fitting 5 folds for each of 10 candidates, totalling 50 fits

Best parameters: {'logreg__C': 0.1, 'logreg__penalty': 'l2', 'logreg__solver': 'liblinear'}
Best cross-validation accuracy: 0.7946

Cross-validation results:
Params: {'logreg__C': 0.01, 'logreg__penalty': 'l1', 'logreg__solver': 'liblinear'}
  Accuracy: 0.7868 (+/- 0.0188)
  F1: 0.7094 (+/- 0.0316)
  ROC AUC: 0.8333 (+/- 0.0258)
Params: {'logreg__C': 0.01, 'logreg__penalty': 'l2', 'logreg__solver': 'liblinear'}
  Accuracy: 0.7868 (+/- 0.0177)
  F1: 0.7178 (+/- 0.0241)
  ROC AUC: 0.8523 (+/- 0.0205)
Params: {'logreg__C': 0.1, 'logreg__penalty': 'l1', 'logreg__solver': 'liblinear'}
  Accuracy: 0.7902 (+/- 0.0215)
  F1: 0.7205 (+/- 0.0256)
  ROC AUC: 0.8520 (+/- 0.0235)
Params: {'logreg__C': 0.1, 'logreg__penalty': 'l2', 'logreg__solver': 'liblinear'}
  Accuracy: 0.7946 (+/- 0.0187)
  F1: 0.7247 (+/- 0.0236)
  ROC AUC: 0.8534 (+/- 0.0223)
Params: {'logreg__C': 1, 'logreg__penalty': 'l1', 'log

In [138]:
# Model tốt nhất
best_model_lr = grid_search.best_estimator_

# Đánh giá trên training set
y_pred_train = best_model_lr.predict(X_train)
y_pred_proba_train = best_model_lr.predict_proba(X_train)[:, 1]  # Xác suất lớp 1

# Tính các metrics
train_accuracy = best_model_lr.score(X_train, y_train)
train_f1 = f1_score(y_train, y_pred_train)
train_roc_auc = roc_auc_score(y_train, y_pred_proba_train)

print(f"\n=== KẾT QUẢ TRÊN TRAINING SET ===")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"F1 Score: {train_f1:.4f}")
print(f"ROC AUC: {train_roc_auc:.4f}")

# print("\nClassification Report:")
# print(classification_report(y_train, y_pred_train))


=== KẾT QUẢ TRÊN TRAINING SET ===
Accuracy: 0.8025
F1 Score: 0.7333
ROC AUC: 0.8575


## Dump mô hình logistic regression


In [140]:
import joblib

# Lưu model
joblib.dump(best_model_lr, '../../exps/trainbase_18102025/logistic_regression_model.pkl')
print("Model đã được lưu bằng joblib")

# Tham số
feature_names = X_train.columns.tolist()
scaler = best_model_lr.named_steps['scaler']

# Lưu nhiều objects cùng lúc
joblib.dump({
    'model': best_model_lr,
    'feature_names': feature_names,  # Nếu có
    'scaler': scaler,  # Nếu có
    'training_date': '2024-10-18'
}, '../../exps/trainbase_18102025/model_package.pkl')

# Load lại model
loaded_model = joblib.load('../../exps/trainbase_18102025/logistic_regression_model.pkl')

Model đã được lưu bằng joblib


# Random forest


In [128]:
# Tạo pipeline cho Random Forest
pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),  # Vẫn có thể chuẩn hóa cho RF (không bắt buộc)
    ('rf', RandomForestClassifier(random_state=42))
])

# Thiết lập hyperparameters cho Random Forest
param_grid_rf = {
    'rf__n_estimators': [50, 100, 200],           # Số cây trong rừng
    'rf__max_depth': [3, 5, 7, None],             # Độ sâu tối đa của cây
    'rf__min_samples_split': [2, 5, 10],          # Số mẫu tối thiểu để split
    'rf__min_samples_leaf': [1, 2, 4],            # Số mẫu tối thiểu ở lá
    'rf__max_features': ['sqrt', 'log2', None]    # Số features xét tại mỗi split
}

# Sử dụng Stratified K-Fold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search với multiple scoring metrics
grid_search_rf = GridSearchCV(
    pipeline_rf, 
    param_grid_rf, 
    cv=cv, 
    scoring={'accuracy': 'accuracy', 'f1': 'f1', 'roc_auc': 'roc_auc'},
    refit='accuracy',  # Chọn metric để chọn best model
    n_jobs=-1,
    verbose=1
)

# Train model
print("\nBắt đầu training Random Forest với GridSearch...")
grid_search_rf.fit(X_train, y_train)

# Kết quả
print("\nBest parameters:", grid_search_rf.best_params_)
print("Best cross-validation accuracy: {:.4f}".format(grid_search_rf.best_score_))

# Lấy tất cả kết quả cross-validation
cv_results = grid_search_rf.cv_results_
print("\nCross-validation results (top 5 combinations):")
# Chỉ hiển thị 5 combination tốt nhất
best_indices = np.argsort(cv_results['mean_test_accuracy'])[-5:][::-1]

for i in best_indices:
    print(f"\nParams: {cv_results['params'][i]}")
    print(f"  Accuracy: {cv_results['mean_test_accuracy'][i]:.4f} (+/- {cv_results['std_test_accuracy'][i]:.4f})")
    print(f"  F1: {cv_results['mean_test_f1'][i]:.4f} (+/- {cv_results['std_test_f1'][i]:.4f})")
    print(f"  ROC AUC: {cv_results['mean_test_roc_auc'][i]:.4f} (+/- {cv_results['std_test_roc_auc'][i]:.4f})")

# Model tốt nhất
best_model_rf = grid_search_rf.best_estimator_

# Đánh giá trên training set
y_pred_train = best_model_rf.predict(X_train)
y_pred_proba_train = best_model_rf.predict_proba(X_train)[:, 1]  # Xác suất lớp 1

# Tính các metrics
train_accuracy = best_model_rf.score(X_train, y_train)
train_f1 = f1_score(y_train, y_pred_train)
train_roc_auc = roc_auc_score(y_train, y_pred_proba_train)


Bắt đầu training Random Forest với GridSearch...
Fitting 5 folds for each of 324 candidates, totalling 1620 fits

Best parameters: {'rf__max_depth': 7, 'rf__max_features': None, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 5, 'rf__n_estimators': 50}
Best cross-validation accuracy: 0.8440

Cross-validation results (top 5 combinations):

Params: {'rf__max_depth': 7, 'rf__max_features': None, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 5, 'rf__n_estimators': 50}
  Accuracy: 0.8440 (+/- 0.0165)
  F1: 0.7776 (+/- 0.0294)
  ROC AUC: 0.8761 (+/- 0.0175)

Params: {'rf__max_depth': None, 'rf__max_features': 'sqrt', 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 5, 'rf__n_estimators': 50}
  Accuracy: 0.8440 (+/- 0.0077)
  F1: 0.7867 (+/- 0.0160)
  ROC AUC: 0.8808 (+/- 0.0196)

Params: {'rf__max_depth': None, 'rf__max_features': 'log2', 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 5, 'rf__n_estimators': 50}
  Accuracy: 0.8440 (+/- 0.0077)
  F1: 0.7867 (+/- 0.0160)
  RO

In [129]:
print(f"\n=== KẾT QUẢ TRÊN TRAINING SET ===")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"F1 Score: {train_f1:.4f}")
print(f"ROC AUC: {train_roc_auc:.4f}")

# print("\nClassification Report:")
# print(classification_report(y_train, y_pred_train))


=== KẾT QUẢ TRÊN TRAINING SET ===
Accuracy: 0.8956
F1 Score: 0.8540
ROC AUC: 0.9527


# Dump mô hình Random Forest


In [137]:
import joblib

# Lưu model
joblib.dump(best_model_rf, '../../exps/trainbase_18102025/random_forest_model.pkl')
print("Model đã được lưu bằng joblib")

# Tham số
feature_names = X_train.columns.tolist()
scaler = best_model_rf.named_steps['scaler']

# Lưu nhiều objects cùng lúc
joblib.dump({
    'model': best_model_rf,
    'feature_names': feature_names,  # Nếu có
    'scaler': scaler,  # Nếu có
    'training_date': '2024-10-18'
}, '../../exps/trainbase_18102025/model_package.pkl')

# Load lại model
loaded_model = joblib.load('../../exps/trainbase_18102025/random_forest_model.pkl')

Model đã được lưu bằng joblib


# XGBoost


In [131]:
# Tạo pipeline cho XGBoost
pipeline_xgb = Pipeline([
    ('scaler', StandardScaler()),  # XGBoost thường không cần scaling nhưng vẫn có thể dùng
    ('xgb', XGBClassifier(random_state=42, eval_metric='logloss'))
])

# Thiết lập hyperparameters cho XGBoost
param_grid_xgb = {
    'xgb__n_estimators': [50, 100, 200],           # Số cây (boosting rounds)
    'xgb__max_depth': [3, 5, 7],                   # Độ sâu tối đa của cây
    'xgb__learning_rate': [0.01, 0.1, 0.2],       # Tốc độ học
    'xgb__subsample': [0.8, 0.9, 1.0],            # Tỷ lệ mẫu cho mỗi cây
    'xgb__colsample_bytree': [0.8, 0.9, 1.0],     # Tỷ lệ features cho mỗi cây
    'xgb__reg_alpha': [0, 0.1, 1],                # L1 regularization
    'xgb__reg_lambda': [1, 10, 100]               # L2 regularization
}

# Sử dụng Stratified K-Fold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search với multiple scoring metrics
grid_search_xgb = GridSearchCV(
    pipeline_xgb, 
    param_grid_xgb, 
    cv=cv, 
    scoring={'accuracy': 'accuracy', 'f1': 'f1', 'roc_auc': 'roc_auc'},
    refit='accuracy',  # Chọn metric để chọn best model
    n_jobs=-1,
    verbose=1
)

# Train model
print("\nBắt đầu training XGBoost với GridSearch...")
grid_search_xgb.fit(X_train, y_train)

# Kết quả
print("\nBest parameters:", grid_search_xgb.best_params_)
print("Best cross-validation accuracy: {:.4f}".format(grid_search_xgb.best_score_))

# Lấy tất cả kết quả cross-validation
cv_results = grid_search_xgb.cv_results_
print("\nCross-validation results (top 5 combinations):")
# Chỉ hiển thị 5 combination tốt nhất
best_indices = np.argsort(cv_results['mean_test_accuracy'])[-5:][::-1]

for i in best_indices:
    print(f"\nParams: {cv_results['params'][i]}")
    print(f"  Accuracy: {cv_results['mean_test_accuracy'][i]:.4f} (+/- {cv_results['std_test_accuracy'][i]:.4f})")
    print(f"  F1: {cv_results['mean_test_f1'][i]:.4f} (+/- {cv_results['std_test_f1'][i]:.4f})")
    print(f"  ROC AUC: {cv_results['mean_test_roc_auc'][i]:.4f} (+/- {cv_results['std_test_roc_auc'][i]:.4f})")

# Model tốt nhất
best_model_xgb = grid_search_xgb.best_estimator_

# Đánh giá trên training set
y_pred_train = best_model_xgb.predict(X_train)
y_pred_proba_train = best_model_xgb.predict_proba(X_train)[:, 1]  # Xác suất lớp 1

# Tính các metrics
train_accuracy = best_model_xgb.score(X_train, y_train)
train_f1 = f1_score(y_train, y_pred_train)
train_roc_auc = roc_auc_score(y_train, y_pred_proba_train)


Bắt đầu training XGBoost với GridSearch...
Fitting 5 folds for each of 2187 candidates, totalling 10935 fits

Best parameters: {'xgb__colsample_bytree': 1.0, 'xgb__learning_rate': 0.2, 'xgb__max_depth': 5, 'xgb__n_estimators': 50, 'xgb__reg_alpha': 1, 'xgb__reg_lambda': 1, 'xgb__subsample': 0.8}
Best cross-validation accuracy: 0.8473

Cross-validation results (top 5 combinations):

Params: {'xgb__colsample_bytree': 1.0, 'xgb__learning_rate': 0.2, 'xgb__max_depth': 5, 'xgb__n_estimators': 50, 'xgb__reg_alpha': 1, 'xgb__reg_lambda': 1, 'xgb__subsample': 0.8}
  Accuracy: 0.8473 (+/- 0.0293)
  F1: 0.7890 (+/- 0.0431)
  ROC AUC: 0.8827 (+/- 0.0208)

Params: {'xgb__colsample_bytree': 1.0, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 5, 'xgb__n_estimators': 50, 'xgb__reg_alpha': 0, 'xgb__reg_lambda': 1, 'xgb__subsample': 0.8}
  Accuracy: 0.8462 (+/- 0.0261)
  F1: 0.7850 (+/- 0.0399)
  ROC AUC: 0.8830 (+/- 0.0217)

Params: {'xgb__colsample_bytree': 0.8, 'xgb__learning_rate': 0.1, 'xgb__max_de

In [ ]:

print(f"\n=== KẾT QUẢ TRÊN TRAINING SET ===")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"F1 Score: {train_f1:.4f}")
print(f"ROC AUC: {train_roc_auc:.4f}")

# print("\nClassification Report:")
# print(classification_report(y_train, y_pred_train))


=== KẾT QUẢ TRÊN TRAINING SET ===
Accuracy: 0.8967
F1 Score: 0.8589
ROC AUC: 0.9585

Classification Report:
              precision    recall  f1-score   support

         0.0       0.89      0.95      0.92       549
         1.0       0.90      0.82      0.86       342

    accuracy                           0.90       891
   macro avg       0.90      0.88      0.89       891
weighted avg       0.90      0.90      0.90       891



# Chọn ra mô hình tối ưu


In [134]:
# Sử dụng best model từ GridSearch
best_model = best_model_xgb

# Kiểm tra model đã được train chưa
print("Model đã được train:", hasattr(best_model.named_steps['xgb'], 'feature_importances_'))

# Dự đoán trên test set
test_preds = best_model.predict(X_test)
test_probas = best_model.predict_proba(X_test)[:, 1]  # Nếu cần xác suất

print(f"Số lượng predictions: {len(test_preds)}")
print(f"Phân bố predictions: {pd.Series(test_preds).value_counts().sort_index()}")

test_data_orig = pd.read_csv("../../../data/test.csv")

# Tạo submission
submission = pd.DataFrame({
    "PassengerId": test_data_orig["PassengerId"],
    "Survived": test_preds
})

# Lưu submission
submission.to_csv("../../exps/trainbase_18102025/submission_xgboost.csv", index=False)
print("Submission file đã được lưu!")

# Kiểm tra submission
print("\n=== SUBMISSION INFO ===")
print(f"Shape: {submission.shape}")
print(f"PassengerId range: {submission['PassengerId'].min()} - {submission['PassengerId'].max()}")
print(f"Survived distribution:\n{submission['Survived'].value_counts().sort_index()}")

Model đã được train: True
Số lượng predictions: 418
Phân bố predictions: 0    283
1    135
Name: count, dtype: int64
Submission file đã được lưu!

=== SUBMISSION INFO ===
Shape: (418, 2)
PassengerId range: 892 - 1309
Survived distribution:
Survived
0    283
1    135
Name: count, dtype: int64


# Dump mô hình XGBoost


In [136]:
import joblib

# Lưu model
joblib.dump(best_model_xgb, '../../exps/trainbase_18102025/xgboost_model.pkl')
print("Model đã được lưu bằng joblib")

# Tham số
feature_names = X_train.columns.tolist()
scaler = best_model_xgb.named_steps['scaler']

# Lưu nhiều objects cùng lúc
joblib.dump({
    'model': best_model_xgb,
    'feature_names': feature_names,  # Nếu có
    'scaler': scaler,  # Nếu có
    'training_date': '2024-10-18'
}, '../../exps/trainbase_18102025/model_package.pkl')

# Load lại model
loaded_model = joblib.load('../../exps/trainbase_18102025/xgboost_model.pkl')

Model đã được lưu bằng joblib
